In [3]:
import numpy as np
import os

## Output File Processing

### Reading in output file and sectioning off multiplicities and roots

In [5]:
with open('orca.out', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        if 'QD-NEVPT2 Results' in line:
            Results_Index = i
        if 'QD-NEVPT2 TOTAL ENERGIES' in line:
            TransitionEnergies_Index = i
    DataLines = lines[Results_Index:TransitionEnergies_Index]

for j, line in enumerate(DataLines):
    if 'MULT 4' in line:
        Quartet_Index = j
    if 'MULT 2' in line:
        QuartetEnd_Index = j-2
        Doublet_Index = j
        DoubletEnd_Index = len(DataLines)-1

QuartetData = DataLines[Quartet_Index:QuartetEnd_Index]
DoubletData = DataLines[Doublet_Index:DoubletEnd_Index]

#Section off output file by roots
for k, line in enumerate(QuartetData):
    if 'ROOT = 0' in line:
        QuartetRoot0_Index = k
        QuartetRoot0Data = QuartetData[QuartetRoot0_Index:]

    if 'ROOT = 1' in line:
        QuartetRoot0End_Index = k-2
        QuartetRoot1_Index = k
        QuartetRoot0Data = QuartetData[QuartetRoot0_Index:QuartetRoot0End_Index]
        QuartetRoot1Data = QuartetData[QuartetRoot1_Index:]
        
    if 'ROOT = 2' in line:
        QuartetRoot1End_Index = k-2
        QuartetRoot2_Index = k
        QuartetRoot1Data = QuartetData[QuartetRoot1_Index:QuartetRoot1End_Index]
        QuartetRoot2Data = QuartetData[QuartetRoot2_Index:]
        

for p, line in enumerate(DoubletData):
    if 'ROOT = 0' in line:
        DoubletRoot0_Index = p
        DoubletRoot0Data = DoubletData[DoubletRoot0_Index:]
    
    if 'ROOT = 1' in line:
        DoubletRoot0End_Index = p-2
        DoubletRoot1_Index = p
        DoubletRoot0Data = DoubletData[DoubletRoot0_Index:DoubletRoot0End_Index]
        DoubletRoot1Data = DoubletData[DoubletRoot1_Index:] 
        
                
    if 'ROOT = 2' in line:
        DoubletRoot1End_Index = p-2
        DoubletRoot2_Index = p
        DoubletRoot1Data = DoubletData[DoubletRoot1_Index:DoubletRoot1End_Index]
        DoubletRoot2Data = DoubletData[DoubletRoot2_Index:]
    
    if 'ROOT = 3' in line:
        DoubletRoot2End_Index = p-2
        DoubletRoot3_Index = p
        DoubletRoot2Data = DoubletData[DoubletRoot2_Index:DoubletRoot2End_Index]
        DoubletRoot3Data = DoubletData[DoubletRoot3_Index:]
    
    if 'ROOT = 4' in line:
        DoubletRoot3End_Index = p-2
        DoubletRoot4_Index = p
        DoubletRoot3Data = DoubletData[DoubletRoot3_Index:DoubletRoot3End_Index]
        DoubletRoot4Data = DoubletData[DoubletRoot4_Index:]
        

### Project and normalize the basis

In [10]:
def NormalizeAndProject(RootData):
    Coefficients = []
    for i in RootData:
        if '[' in i:
            Coefficients.append(i)
    Coefficients_Projected = []
    Coefficients_NormProj = []
    ProjCoeffValuesArray = []
    NormProjCoeffValuesArray = []
    for coeffline in Coefficients:
        if 'duu' or 'udu' or 'uud' or 'uuu' in coeffline:
            Coefficients_Projected.append(coeffline)
            words = coeffline.split()
            CoeffValue = float(words[1])
            ProjCoeffValuesArray.append(CoeffValue)
    for i in range(len(ProjCoeffValuesArray)):
        NormProjCoeffValue = np.sqrt((np.square(ProjCoeffValuesArray[i]))/(np.sum(np.square(ProjCoeffValuesArray))))
        NormProjCoeffValuesArray.append(NormProjCoeffValue)
    for t,projcoeffline in enumerate(Coefficients_Projected):
        words = projcoeffline.split()
        spinID = words[0]
        spinCoeff = NormProjCoeffValuesArray[t]
        Coefficients_NormProj.append(F'[{spinID}] : {spinCoeff}')
    return Coefficients, Coefficients_Projected, ProjCoeffValuesArray, NormProjCoeffValuesArray, Coefficients_NormProj 

**Try with root with only neutral determinants**

In [11]:
DR1Coefficients, DR1Coefficients_Projected, DR1ProjCoeffValuesArray, DR1NormProjCoeffValuesArray, DR1Coefficients_NormProj = NormalizeAndProject(DoubletRoot1Data)

print(DR1Coefficients)
print(DR1Coefficients_Projected)
print(DR1ProjCoeffValuesArray)
print(DR1NormProjCoeffValuesArray)
print(DR1Coefficients_NormProj)

['   [duu]     -0.816496570\n', '   [udu]      0.408246839\n', '   [uud]      0.408249731\n']
['   [duu]     -0.816496570\n', '   [udu]      0.408246839\n', '   [uud]      0.408249731\n']
[-0.81649657, 0.408246839, 0.408249731]
[0.8164965809260187, 0.40824684446299003, 0.40824973646302876]
['[[duu]] : 0.8164965809260187', '[[udu]] : 0.40824684446299003', '[[uud]] : 0.40824973646302876']


**Try with root with both ionic and neutral determinants**

## Writing Input File

### Heading